In [36]:
import random
import string

''.join((random.choice(string.ascii_lowercase) for x in range(6)))

'gxobrt'

In [45]:
random_string = ''.join(str(random.randint(0, 9)) for i in range(6))
random_string

'616245'

In [47]:
users = []
for i in range(100):
    username = ''.join((random.choice(string.ascii_lowercase) for x in range(6)))
    password = ''.join(str(random.randint(0, 9)) for i in range(6))
    users.append((username, password))
l = len(users)
batch_size = l//10
for i in range(0, len(users), batch_size):
    batch = users[i:i + batch_size]
    print(batch)

[('rhzfds', '678064'), ('qqjjvf', '555458'), ('bwfsud', '545464'), ('antksa', '500091'), ('mkkzjb', '625591'), ('eotqnf', '222252'), ('bxijpi', '605910'), ('pgzsbk', '583618'), ('piexra', '562131'), ('mktvur', '486517')]
[('wghtyk', '056674'), ('tcrekd', '960405'), ('hjithe', '698634'), ('suivlp', '148404'), ('fxqpzp', '236287'), ('xgbfdu', '527531'), ('cdpkos', '838869'), ('yheplo', '747082'), ('shpbjp', '766529'), ('nxycza', '978316')]
[('khtcqn', '322642'), ('jjunvz', '478752'), ('vahnkv', '840132'), ('cfiwfa', '627871'), ('ijofma', '322524'), ('ioitrr', '293633'), ('zqumwr', '966930'), ('vutgwo', '528936'), ('oiamtc', '176972'), ('bjmnfu', '094640')]
[('qkmhnv', '981597'), ('cjbsly', '390720'), ('vmzbit', '136263'), ('fcpojq', '408283'), ('fveatz', '776640'), ('mxvgrk', '129082'), ('ydrchm', '417280'), ('qqjroq', '526533'), ('xjosqa', '225954'), ('yuueol', '787964')]
[('qoqiir', '860713'), ('swaxce', '877376'), ('gsuinh', '760015'), ('kaclom', '344358'), ('xaaluz', '721010'), ('nkw

In [7]:
from __future__ import print_function
import mysql.connector
from mysql.connector import errorcode

from flask import Flask, request
from flask_limiter import Limiter 
from hashlib import sha256
import random
import string
import time
import json

In [15]:
cnx = mysql.connector.connect(user='root', password='QuocAnh24102001', database='hehe')
cursor = cnx.cursor()
cursor.execute("SELECT * FROM users WHERE username = 'xkkcrf'")
data = cursor.fetchone()
if data != None:
    userId, username, password, passw, loggedIn, loggedAt = data
    print(username)

else:
    print('no')

xkkcrf


In [10]:
# idea 
hashed_arr = []
for i in range(1000000):
    str_i = str(i).rjust(6, '0')
    hashed_arr.append(sha256(str_i.encode('utf-8')).hexdigest())
start_time = time.time()
hashed = '2fc49e276459565ff3be8254f40ea0c72b181d92112bdfc5498b00d55b95e348'
hashed_arr.index(hashed)
end_time = time.time()
print("Total insertion time:", end_time - start_time, "seconds")


Total insertion time: 0.01698923110961914 seconds
